# Capstone Project - The Battle of the Neighbourhoods

## Introduction: Business Problem

In this project we will try to find an optimal location for a bakery in/around Downtown Toronto, Canada.

Because of the trendiness of bakeries in Toronto we will try to detect locations that are not already crowded with bakeries but yet as close to city centre as possible.

Based on these criteria, we will use our data science prowess to generate a few promissing streets and spotts. Advantages of each area will then be clearly expressed for the purposes of finding the best possible final location which can be chosen by stakeholders.

## Data

Based on our problem as defined above, factors that will influence our decission are:

    - number of existing bakeries in/around the downtown area;
    - distance to bakeries in the downtown area, if any distance of neighborhood from city centre.

We decided to use regularly spaced grid of locations, centered around city center, to define our neighbourhoods.

Data sources that will be needed to extract the required information as follows:

    - centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding;
    - number of bakeries and their type and location in every neighborhood in and around Downtown Toronto will be obtained using Foursquare API;
    - coordinates of Downtown Toronto will be obtained using Google Maps API geocoding.
